In [ ]:
%pip install -q -r requirements.txt

In [ ]:
%load_ext autoreload
%autoreload 1
from imports import *
from functions import *

Connection to MySQL

In [ ]:
mysql_user = "flosrv"
password = "Nesrine123"
host = "localhost"
port = 3306
database = "Oceanography_data_analysis"
metadata = MetaData()
# Connect to the database
engine = create_engine(f"mysql+mysqlconnector://{mysql_user}:{password}@{host}/{database}", isolation_level ='AUTOCOMMIT')

Get Available Stations ID List

In [ ]:
# get all stations and some metadata as a Pandas DataFrame
stations_df = api.stations()
# parse the response as a dictionary
stations_df = api.stations(as_df=True)

print(len(stations_df))

Filter Buoys by Remarks

In [ ]:
access_error_url_list = []

# Liste de mots à rechercher dans la colonne "Remark"
blacklist = ["Failure", "ceased", "failed", "recovered", "stopped", 'adrift']
stations_id_set = set()

print(f'Avant Filtre: {stations_df.shape[0]}')

# Liste pour collecter les indices à supprimer
indices_a_supprimer = []

# Parcours des lignes de la DataFrame
for idx, row in stations_df.iterrows():
    station_id = row["Station"]
    station_Location = row["Hull No./Config and Location"]  # Extraire la valeur de la cellule pour chaque ligne
    
    # Extraction du nom de la station si un ")" est trouvé
    if ")" in station_Location:
        station_name = station_Location.split(')')[1].rstrip(" )")  # On enlève l'espace et la parenthèse en fin de chaîne
    else:
        station_name = station_Location.strip()  # Si pas de ")", on garde toute la chaîne

    station_name = station_name.rstrip(" )").replace("(", "").replace(")", "").strip()

    # Nettoyage final pour enlever toute parenthèse ou espace en fin de nom
    station_name = station_name.rstrip(" )")

    # Vérifier si "Remark" n'est pas NaN et si un des éléments de blacklist est dans "Remark"
    if isinstance(row["Remark"], str) and any(blacklist_word.lower() in row["Remark"].lower() for blacklist_word in blacklist):
        # Ajouter l'index à la liste
        indices_a_supprimer.append(idx)
        print(f'Buoy {station_id} Failed the Remark Test !')
        url = get_buoy_url(station_id)
        access_error_url_list.append(url)
    else:
        print(f'Buoy {station_id} Passed the Remark Test !')

# Supprimer les lignes après la boucle
stations_df.drop(index=indices_a_supprimer, inplace=True)

print(f'Après Filtre: {stations_df.shape[0]}')

Build Buoys_datas Dict

In [ ]:
# Dictionnaire pour stocker les DataFrames, clé : ID de la bouée, valeur : DataFrame
buoy_datas = {}
buoy_list = []

# Parcours de chaque bouée dans stations_df
for index, row in stations_df.iterrows():
    buoy_id = row['Station']
    metadata = get_station_metadata(buoy_id)

    # ✅ Récupérer les données sous forme de dictionnaire
    buoy_info = parse_buoy_json(metadata)

    # ✅ Stocker directement les données dans buoy_datas
    buoy_datas[buoy_id] = buoy_info
    buoy_list.append(buoy_id)

# Affichage du nombre de bouées réussies et échouées
print(f"Nombre de bouées traitées : {len(buoy_datas)}\n")

# Afficher le contenu de buoy_datas

first_key =next(iter(buoy_datas))
first_key
buoy_datas[first_key]

In [ ]:
buoy_datas

In [ ]:
# df_buoys_data = pd.DataFrame(buoy_datas)
# df_buoys_data.head()

In [22]:
# 🚀 Démarrage du processus
print("\n🚀 Démarrage du processus de collecte et d'insertion des données...\n")

# Initialisation des compteurs
marine_data_collected_successfully = marine_data_collected_failed = 0
marine_data_inserted_successfully = marine_data_inserted_failed = 0
meteo_data_collected_successfully = meteo_data_collected_failed = 0
meteo_data_inserted_successfully = meteo_data_inserted_failed = 0

success = False
total_stations = stations_df.shape[0]
count = 0

# 🔄 Parcours des bouées / stations
for idx, row in stations_df.iterrows():
    buoy_id = row["Station"]

    ######### 🌊 MARINE DATA #########
    try:
        df_marine = NDBC.realtime_observations(buoy_id)
        if df_marine is None or df_marine.empty:
            marine_data_collected_failed += 1
            continue

        marine_data_collected_successfully += 1
    except Exception as e:
        print(f"⚠️ Erreur collecte marine {buoy_id}: {e}")
        marine_data_collected_failed += 1
        continue

    # Ajout des métadonnées
    try:
        buoy_info = buoy_datas.get(buoy_id, {})
        Lat, Lon = buoy_info.get('lat_buoy'), buoy_info.get('lon_buoy')
        if Lat is None or Lon is None:
            raise ValueError(f"Données manquantes pour {buoy_id}")

        df_marine['Lat'] = Lat
        df_marine['Lon'] = Lon
        df_marine['Water_depth'] = buoy_info.get('Water_depth', None)
        df_marine.columns = ['Datetime' if 'date' in col.lower() or 'time' in col.lower() else col for col in df_marine.columns]
        df_marine['Datetime'] = df_marine['Datetime'].dt.tz_localize(None)

        buoy_datas[buoy_id]["Bronze Marine DataFrame"] = df_marine

        station_zone = safe_get(parse_buoy_json(get_station_metadata(buoy_id)), "station_zone")
        Bronze_Marine_table_Name = f"br_{buoy_id}_marine_{station_zone}".replace('.', '_').replace('-', '_').replace(' ', '_').lower()

    except Exception as e:
        print(f"⚠️ Erreur métadonnées marine {buoy_id}: {e}")
        marine_data_collected_failed += 1
        continue

    # Création de la table et insertion des nouvelles lignes
    try:
        
        create_table_in_mysql(engine=engine, df=df_marine, table_name=Bronze_Marine_table_Name)
        insert_new_rows(engine=engine, df=df_marine, table_name=Bronze_Marine_table_Name)
        marine_data_inserted_successfully += 1
    except Exception as e:
        print(f"❌💾 Erreur insertion marine {buoy_id}: {e}")
        marine_data_inserted_failed += 1
        continue

    ######### ⛅ METEO DATA #########
    try:
        df_meteo = meteo_api_request([Lat, Lon])
        if df_meteo is None or df_meteo.empty:
            meteo_data_collected_failed += 1
            continue
        
        
        rename_columns(df_meteo, {'date':'Datetime'})
        df_meteo.columns = ['Datetime' if 'date' in col.lower() or 'time' in col.lower() else col for col in df_meteo.columns]
        df_meteo['Datetime'] = df_meteo['Datetime'].dt.tz_localize(None)
        buoy_datas[buoy_id]["Bronze Meteo DataFrame"] = df_meteo
        meteo_data_collected_successfully += 1
    except Exception as e:
        print(f"⚠️ Erreur collecte météo {buoy_id}: {e}")
        meteo_data_collected_failed += 1
        continue

    # Création de la table et insertion des nouvelles lignes
    try:
        Bronze_Meteo_table_Name = f"br_{buoy_id}_meteo_{station_zone}".replace('.', '_').replace(' ', '_').replace('-', '_').lower()
        create_table_in_mysql(engine=engine, df=df_meteo, table_name=Bronze_Meteo_table_Name)
        insert_new_rows(engine=engine, df=df_meteo, table_name=Bronze_Meteo_table_Name)
        meteo_data_inserted_successfully += 1
    except Exception as e:
        print(f"❌💾 Erreur insertion météo {buoy_id}: {e}")
        meteo_data_inserted_failed += 1
        continue
try:
    df_buoy_datas = pd.DataFrame.from_dict(buoy_datas, orient='index')
    create_table_in_mysql(engine=engine, df=df_buoy_datas, table_name="buoys_data")
except Exception as e:
    print(f"\nFailed to load Buoy Datas DataFrame:\n{e}")
# 🔚 Résumé final

print("\n📝 Résumé final :")
print(f"🌊 Marine - Collecte ✅ {marine_data_collected_successfully} ❌ {marine_data_collected_failed}")
print(f"🌊 Marine - Insertion ✅ {marine_data_inserted_successfully} ❌ {marine_data_inserted_failed}")
print(f"⛅ Météo - Collecte ✅ {meteo_data_collected_successfully} ❌ {meteo_data_collected_failed}")
print(f"⛅ Météo - Insertion ✅ {meteo_data_inserted_successfully} ❌ {meteo_data_inserted_failed}")



🚀 Démarrage du processus de collecte et d'insertion des données...


🔍 Début du parsing de la bouée...
🌍 Zone de la station : grays reef
🆔 Station ID : 41008
✅ Coordonnées extraites : Latitude = 31.40N, Longitude = 80.87W
🌊 Water Depth : 16 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation : 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41008
✅ Parsing terminé !

📊 station_zone : grays reef
⚠️ La table 'br_41008_marine_grays_reef' existe déjà dans MySQL.
✅ Aucune nouvelle ligne à insérer dans 'br_41008_marine_grays_reef'.
⚠️ La table 'br_41008_meteo_grays_reef' existe déjà dans MySQL.
✅ Aucune nouvelle ligne à insérer dans 'br_41008_meteo_grays_reef'.

🔍 Début du parsing de la bouée...
🌍 Zone de la station : ne st martin
🆔 Station ID : 41044
✅ Coordonnées extraites : Latitude = 21.58N, Longitude = 58.63W
🌊 Water Depth : 5419 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation : 2.4
💨 Anemometer Height : 3.8
🌤️ Air

In [ ]:
# # Convertir le dictionnaire en DataFrame, en utilisant 'buoy_id' comme index
# df_buoy_datas = pd.DataFrame.from_dict(buoy_datas, orient='index')

# # Réinitialiser l'index et renommer la colonne
# df_buoy_datas = df_buoy_datas.reset_index(drop=False)
# df_buoy_datas.rename(columns={"index": "Station ID"}, inplace=True)

# table_name="buoy_datas"
# # Charger les données dans la table
# try:
#     load_data_in_table_sqlalchemy(conn=conn, schema=schema, df=df_buoy_datas, table_name=table_name, key_column="Station ID")
#     print(f"Données chargées avec succès dans la table {table_name}.")
# except Exception as e:
#     print(f"Erreur lors du chargement des données : {e}")

# # Affichage du DataFrame
# df_buoy_datas.head()

Map Visualization

In [ ]:
# list_coords=[]

# for id in buoy_list:
#     metadata = get_station_metadata(id)
    
#     station_id, station_zone, lat_buoy, lon_buoy = parse_buoy_json(metadata)
#     lat_buoy, lon_buoy = convert_coordinates(lat_buoy,lon_buoy)
#     coords = [station_id, station_zone, lat_buoy, lon_buoy]
#     list_coords.append(coords)

# def barycentre(coords):
#     x_coords = [item[2] for item in coords]
#     y_coords = [item[3] for item in coords]

#     x_barycentre = sum(x_coords) / len(coords)
#     y_barycentre = sum(y_coords) / len(coords)

#     return [x_barycentre, y_barycentre]

# # Exemple d'utilisation
# coords = [[1, 2], [3, 4], [5, 6], [7, 8]]
# centre = barycentre(list_coords)
# print(list_coords)
# print("Coordonnées du barycentre :", centre)

In [ ]:
# map = folium.Map(location=(centre[0],centre[1]), 
#                  tiles="Esri.WorldImagery", 
#                  zoom_start=2.5, attr="Données fournies par Esri")
# for loc in list_coords:
#     lat = loc[2]
#     lon = loc[3]
#     station_id = loc[0]
#     station_zone = loc[1]
#     folium.Marker(
#     location=[lat, lon],
#     popup=f"Station ID: {station_id}\n\nZone: {station_zone}\nLat: {lat}\n\nLon: {lon}",
#     icon=folium.Icon(icon="cloud"),
# ).add_to(map)

# map

Columns Check Tests

In [ ]:
choice = random.choice(buoy_list)
choice

In [ ]:
df_marine_test = NDBC.realtime_observations(choice)
print(f'{df_marine_test.shape[0]} rows \n\n{df_marine_test.isna().sum()}')
df_marine_test.head(1)

Check test API Open-Meteo

In [ ]:
df_meteo_test = meteo_api_request(coordinates=[12, 23])
print(f'{df_meteo_test.shape[0]} rows \n{df_meteo_test.isna().sum()}')
df_meteo_test.head(1)

In [ ]:
metadata_extracted = get_station_metadata(choice)
metadata_extracted

In [ ]:
# def drop_schema_or_tables(conn, schema_name, drop_schema=False):
#     try:
#         if drop_schema:
#             # Supprimer le schéma si drop_schema est True
#             print(f"Dropping schema {schema_name}...")
#             conn.execute(text(f"DROP SCHEMA IF EXISTS \"{schema_name}\" CASCADE;"))
#             conn.commit()
#             print(f"Schema {schema_name} dropped successfully.")
#         else:
#             # Supprimer toutes les tables dans le schéma
#             print(f"Dropping all tables in schema {schema_name}...")
#             conn.execute(text(f"""
#                 DO $$ 
#                 DECLARE
#                     r RECORD;
#                 BEGIN
#                     FOR r IN 
#                         SELECT table_name 
#                         FROM information_schema.tables 
#                         WHERE table_schema = :schema_name
#                     LOOP
#                         EXECUTE 'DROP TABLE IF EXISTS "' || :schema_name || '".' || quote_ident(r.table_name) || ' CASCADE';
#                     END LOOP;
#                 END $$;
#             """), {'schema_name': schema_name})
#             conn.commit()
#             print(f"All tables in schema {schema_name} dropped successfully.")
#     except Exception as e:
#         print(f"An error occurred: {e}")
#         conn.rollback()
#         print("Transaction rolled back.")

# drop_schema_or_tables(conn=conn, schema_name=schema, drop_schema=False)

In [ ]:
# def explore_dict(d, indent=0):
#     """ Fonction récursive pour afficher toute la structure du dictionnaire """
#     for key, value in d.items():
#         print(" " * indent + f"- {key}: {type(value)}")
#         if isinstance(value, dict):
#             explore_dict(value, indent + 4)  # Explorer récursivement avec une indentation
#         elif isinstance(value, list):
#             if len(value) > 0:
#                 print(" " * (indent + 4) + f"Liste ({len(value)} éléments), type du premier élément: {type(value[0])}")
#                 if isinstance(value[0], dict):
#                     explore_dict(value[0], indent + 8)  # Explorer si c'est une liste de dicts
#         else:
#             print(" " * (indent + 4) + f"Valeur: {value}")

# # Récupérer la première clé du dictionnaire
# first_key = next(iter(buoy_datas))
# print(f"Exploration de la première clé: {first_key}\n")

# # Exécuter la fonction sur le premier élément uniquement
# explore_dict({first_key: buoy_datas[first_key]})

In [ ]:
# openmongo_creds = r'C:\Users\f.gionnane\Documents\Data Engineering\Credentials\mongo_creds.json'

# with open(openmongo_creds, 'r') as file:
#     content = json.load(file)
#     mongo_user = content["user"]
#     mongo_password = content["password"]
#     mongo_string = content["connection_string"]

# uri = mongo_string
# # Create a new client and connect to the server
# client = MongoClient(uri, server_api=ServerApi('1'))

# # Send a ping to confirm a successful connection
# try:
#     client.admin.command('ping')
#     print("Pinged your deployment. You successfully connected to MongoDB!")
# except Exception as e:
#     print(e)
#     print(f'{mongo_user}\n{mongo_password}\n{mongo_string}')

In [ ]:
# Charger le fichier GeoJSON depuis l'URL
url = "https://gist.githubusercontent.com/jrrickard/8755532505a40f3b8317/raw/ecd98849d3a5f4502b773b986254f19af3b8d8fb/oceans.json"
geojson_data = requests.get(url).json()

# Créer une carte avec un fond satellite ESRI
m = folium.Map(location=[0, 0], zoom_start=3, tiles="Esri.WorldImagery")

# Ajouter le GeoJSON (les océans) à la carte
folium.GeoJson(geojson_data, name="Oceans").add_to(m)

# Fonction pour générer des coordonnées aléatoires dans l'océan
def random_ocean_coords():
    lat = random.uniform(-60, 60)  # Latitude dans les eaux de l'océan
    lon = random.uniform(-180, 180)  # Longitude dans les eaux de l'océan
    return lat, lon

# Générer un DataFrame avec des coordonnées, températures, hauteur des vagues, vitesse du vent, etc.
data = {
    "Coordinates": [],
    "Temperature (°C)": [],
    "Wave Height (m)": [],
    "Wind Speed (km/h)": []
}

# Générer 10 marqueurs avec des valeurs aléatoires
for _ in range(10):
    lat, lon = random_ocean_coords()
    
    # Générer des valeurs aléatoires pour la température, la hauteur des vagues et la vitesse du vent
    temperature = random.uniform(15, 30)  # Température entre 15 et 30°C
    wave_height = random.uniform(0.5, 5)  # Hauteur des vagues entre 0.5m et 5m
    wind_speed = random.uniform(10, 50)  # Vitesse du vent entre 10 km/h et 50 km/h
    
    # Ajouter ces valeurs dans le DataFrame
    data["Coordinates"].append((lat, lon))
    data["Temperature (°C)"].append(temperature)
    data["Wave Height (m)"].append(wave_height)
    data["Wind Speed (km/h)"].append(wind_speed)

# Créer un DataFrame pandas avec ces données
df = pd.DataFrame(data)

# Ajouter les marqueurs et cercles à la carte avec des couleurs dépendant de la température
for index, row in df.iterrows():
    lat, lon = row["Coordinates"]
    temperature = row["Temperature (°C)"]
    
    # Déterminer la couleur du cercle en fonction de la température
    if temperature < 20:
        fill_color = "blue"
    elif temperature < 25:
        fill_color = "green"
    else:
        fill_color = "red"
    
    # Ajouter un marqueur
    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(m)
    
    # Ajouter un cercle autour du marqueur
    folium.CircleMarker(
        location=[lat, lon],
        radius=15,  # Rayon du cercle
        color="black",
        fill=True,
        fill_color=fill_color,
        fill_opacity=0.5
    ).add_to(m)

# Ajouter une couche de contrôle pour la carte
folium.LayerControl().add_to(m)

# Sauvegarder la carte dans un fichier HTML
m.save("ocean_markers_map.html")

# Affichage de la carte dans l'environnement interactif (si vous êtes dans un environnement Jupyter)


In [ ]:
m